# PyScript - Deployment

### Install WML Python Client

### Specify WML credentials and initialize WML client

In [1]:



wml_credentials_401x86_SVT = {
    "url": "https://cpd-zen.apps.cpd40devops2.cp.fyre.ibm.com",
    "username": "admin",
    "password": "Xx0xiMMUjKFhYre78a65VkPJSCuIJE7e",
    "instance_id": "wml_local",
    "version" : "4.0" ,
    "space_id": "93a8cce8-8292-424e-8f02-795b4d78aa26"
}


wml_credentials =  wml_credentials_401x86_SVT



In [2]:
from ibm_watson_machine_learning import APIClient
client = APIClient(wml_credentials)

DEPRECATED!! Python 3.6 framework is deprecated and will be removed on Jan 20th, 2021. It will be read-only mode starting Nov 20th, 2020. i.e you won't be able to create new assets using this client. Use Python 3.7 instead. For details, see https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


2021-07-21 19:38:24,709 - ibm_watson_machine_learning.wml_client_error - WARNING - Failure during creating new asset. (POST https://api.dataplatform.dev.cloud.ibm.com/v2/assets?version=2020-08-01&space_id=440ada9b-af87-4da8-a9fa-a5450825e260)
Status code: 500, body: {"trace":"3g6qekq35lc0b9e9763hyyxb2","errors":[{"code":"ReservedValue","message":"NGPDL3080E: We encountered a problem. '500 Internal Server Error at https://03f2006b-f00d-4659-b3bf-c3768a87ec43-bluemix.cloudant. Error: unknown_error. Reason: function_clause.'"}]}


In [3]:
space_id = wml_credentials['space_id']

In [4]:
client.set.default_space(space_id)

'SUCCESS'

### Create space and set it as default for rest of the operations in this session

In [5]:
client.set.default_space(space_id)

'SUCCESS'

### Create a software specification and associate package extensions created in earlier steps to the software specification

Get base software spec id

In [5]:
base_sw_spec_id = client.software_specifications.get_uid_by_name("default_py3.7_opence")

# base_sw_spec_id = client.software_specifications.get_uid_by_name("default_py3.8")

base_sw_spec_id

'c2057dd4-f42c-5f77-a02f-72bdbd3282c9'

In [6]:
### Py script code:

In [7]:
%%writefile /tmp/CreateSwSpec_script.py

import sys
import time
import os
# sys.path.append("/opt/ibm/scoring/python/cust-libs")
outfname = os.path.join(os.environ.get('BATCH_OUTPUT_DIR'),"swspec.log")
f = open(outfname, "w")
try:
    
    f.write("wml_util ok")
except Exception as ex:
    f.write("wml_util failed: " + repr(sys.exc_info()[0]))
    f.write("sys path:")
    f.write(sys.path)
   
f.close()
# time.sleep(600)

#

Writing /tmp/CreateSwSpec_script.py


In [8]:
!cat /tmp/CreateSwSpec_script.py


import sys
import time
import os
# sys.path.append("/opt/ibm/scoring/python/cust-libs")
outfname = os.path.join(os.environ.get('BATCH_OUTPUT_DIR'),"swspec.log")
f = open(outfname, "w")
try:
    
    f.write("wml_util ok")
except Exception as ex:
    f.write("wml_util failed: " + repr(sys.exc_info()[0]))
    f.write("sys path:")
    f.write(sys.path)
   
f.close()
# time.sleep(600)

#


In [9]:
meta_props = {
    client.script.ConfigurationMetaNames.NAME: "FVTPyScript",
    client.script.ConfigurationMetaNames.SOFTWARE_SPEC_UID: base_sw_spec_id  # required
}

In [11]:
script_path = "/tmp/CreateSwSpec_script.py"
script_details = client.script.store(meta_props, file_path=script_path)
script_id = client.script.get_uid(script_details)
print("created script ",script_id)

Creating Script asset...
SUCCESS
created script  75d2fcb1-85d5-4c69-9c2d-e65198358bfc


In [12]:
script_details

{'metadata': {'space_id': '440ada9b-af87-4da8-a9fa-a5450825e260',
  'name': 'FVTPyScript',
  'guid': '75d2fcb1-85d5-4c69-9c2d-e65198358bfc',
  'href': 'https://api.dataplatform.dev.cloud.ibm.com/v2/assets/75d2fcb1-85d5-4c69-9c2d-e65198358bfc?space_id=440ada9b-af87-4da8-a9fa-a5450825e260',
  'asset_type': 'script',
  'created_at': '2021-07-21T14:09:56Z',
  'last_updated_at': '2021-07-21T14:09:56Z',
  'description': ''},
 'entity': {'script': {'language': {'name': 'python3'},
   'software_spec': {'base_id': 'c2057dd4-f42c-5f77-a02f-72bdbd3282c9'}}}}

## Deployment 

In [13]:
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "pyscript_batch_dep",
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {'name': 'S' }
}
deployment_details = client.deployments.create(script_id, deployment_meta_props)
deployment_id = client.deployments.get_uid(deployment_details)



#######################################################################################

Synchronous deployment creation for uid: '75d2fcb1-85d5-4c69-9c2d-e65198358bfc' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4aa43ea2-84a8-4f83-95a8-e19a3f8a954d'
------------------------------------------------------------------------------------------------




### RUn job

In [14]:
def poll_async_job(wml_client, job_uid):
    import time
    while True:
        job_status = wml_client.deployments.get_job_status(job_uid)
        print(job_status)
        state = job_status['state']
        if state == 'completed' or 'fail' in state:
            return wml_client.deployments.get_job_details(job_uid)
        time.sleep(5)

In [15]:
job_payload_ref = {
    client.deployments.ScoringMetaNames.OUTPUT_DATA_REFERENCE: {
        'type': 'data_asset',
        'location': { 'name': 'deploy_test_script-out' }
    }
}
job = client.deployments.create_job(deployment_id, meta_props=job_payload_ref)
job_id = client.deployments.get_job_uid(job)


In [16]:
job_details = poll_async_job(client, job_id)

{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '', 'running_at': '', 'state': 'queued'}
{'completed_at': '2021-07-21T14:11:10.181546Z', 'running_at': '2021-07-21T14:11:08.260646Z', 'state': 'completed'}


### clean up

In [17]:
client.deployments.delete(deployment_id)

'SUCCESS'

In [34]:
client.script.delete(script_id)

'SUCCESS'

#### TO download a script

In [19]:
client.script.download("b3d9d99b-55ec-4339-8741-78b0a420a408", filename="fvt-pyscript.py")

Successfully saved data asset content to file: 'fvt-pyscript.py'


'/Users/Shared/Relocated-Items/Security/safe/dev/ngp/standalone/notebooks/fvt-pyscript.py'